In [33]:
import pandas as pd

# Loading the dataset
path = "/Users/gozde/code/g0zzy/stress_sense/raw_data/Data.csv"
data = pd.read_csv(path)

data.drop(columns=["Unnamed: 0"], inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

* Do not do any other preprocessing steps such as lowercasing,  removing stopwords, lemmatization, stemming, etc. as they might remove important context for the model to learn from.

* **Transformers are trained on raw(ish) text**

In [34]:
data.statement = data.statement.apply(lambda x: x.strip())

In [35]:
data.head(10)

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety
5,"every break, you must be nervous, like somethi...",Anxiety
6,"I feel scared, anxious, what can I do? And may...",Anxiety
7,Have you ever felt nervous but didn't know why?,Anxiety
8,"I haven't slept well for 2 days, it's like I'm...",Anxiety
9,"I'm really worried, I want to cry.",Anxiety


In [36]:
data.shape

(51093, 2)

In [39]:
new_df = data[data['status'].isin(['Stress','Anxiety'])].copy()
stressed_posts = new_df['statement'].to_list()

In [40]:
new_df.head(10)

,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety
5,"every break, you must be nervous, like somethi...",Anxiety
6,"I feel scared, anxious, what can I do? And may...",Anxiety
7,Have you ever felt nervous but didn't know why?,Anxiety
8,"I haven't slept well for 2 days, it's like I'm...",Anxiety
9,"I'm really worried, I want to cry.",Anxiety


In [41]:
new_df.shape

(5919, 2)

In [15]:
%pip install sentence-transformers

  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.4 MB/s  0:00:00
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 67.5 MB/s  0:00:01a 0:00:010:00:01:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [sentence-transformers]11/12 

In [43]:
from sentence_transformers import SentenceTransformer

sbert = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sbert.encode(stressed_posts, normalize_embeddings=True, show_progress_bar=True)

Batches: 100%|██████████| 185/185 [00:14<00:00, 12.94it/s]


In [44]:
embeddings

array([[ 0.03366001, -0.03712323,  0.01179676, ..., -0.05401971,
         0.04482312, -0.03369883],
       [ 0.01052009, -0.04578955,  0.04768023, ...,  0.01939357,
        -0.06008147, -0.02400911],
       [ 0.05667384, -0.06692987,  0.0137174 , ...,  0.01170356,
        -0.11973175, -0.05436602],
       ...,
       [ 0.07792854, -0.05168048,  0.05081963, ...,  0.04558662,
        -0.08271255,  0.00072075],
       [ 0.0443574 , -0.108092  ,  0.03242296, ..., -0.00546338,
        -0.0618789 , -0.00842193],
       [ 0.0619358 ,  0.0029512 ,  0.0452257 , ...,  0.00549195,
        -0.07962377,  0.04503339]], shape=(5919, 384), dtype=float32)

In [20]:
%pip install bertopic

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.6 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached llvmlite-0.44.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.8 kB)
  Using cached numba-0.61.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.2/26.2 MB 85.1 MB/s  0:00:00 97.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 82.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 81.1 MB/s  0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.40-cp310-cp310-macosx_15_0_arm64.whl size=756683 sha256=91cc566a7e32d2eba9e4a8f862d1b1e644fae6fd967f255c420c194b5c8c8e5f
  Stored in directory: /Users/gozde/Library/Caches/pip/wheels/54/ba/75/e1883bf767fda8494225c99b81fbf5e977e23459d35e93b301
Successfully built hdbscan
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [bertopic]━━

In [46]:
from bertopic import BERTopic

topic_model = BERTopic(
    min_topic_size=100,      # try 50, 80, 100; pick the one that gives 8–12 coherent topics
    n_gram_range=(1, 2),
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(stressed_posts, embeddings)  # pass precomputed embeddings!
topic_info = topic_model.get_topic_info()
topic_info.head(10)

2025-09-02 23:28:14,797 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-02 23:28:15,982 - BERTopic - Dimensionality - Completed ✓
2025-09-02 23:28:15,983 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-02 23:28:16,180 - BERTopic - Cluster - Completed ✓
2025-09-02 23:28:16,182 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-02 23:28:16,926 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,163,-1_the_and_my_to,"[the, and, my, to, it, of, was, that, in, but]",[(vent) I probably got cavities from grinding ...
1,0,5436,0_and_to_the_my,"[and, to, the, my, of, it, in, that, im, me]",[(TW: mention of suicidal behaviour) anyone wi...
2,1,320,1_restless_is_the_sleep,"[restless, is, the, sleep, to, you, im, and, w...","[restless and restless, want to be angry, Rest..."


In [48]:
# Top words per topic
for tid in topic_info["Topic"].head(10):
    if tid == -1:
        continue
    print("Topic", tid, topic_model.get_topic(tid)[:10])  # list of (word, score)

# Sample 5 docs from a topic
import numpy as np
def sample_topic(tid, k=5):
    idx = np.where(np.array(topics) == tid)[0]
    for i in np.random.choice(idx, size=min(k, len(idx)), replace=False):
        print("-", stressed_posts[i][:200], "...\n")

Topic 0 [('and', np.float64(0.05263530334691668)), ('to', np.float64(0.048808430112600124)), ('the', np.float64(0.04243667215482987)), ('my', np.float64(0.040666190712420364)), ('of', np.float64(0.030950683093939002)), ('it', np.float64(0.02919478569748842)), ('in', np.float64(0.0242363797174038)), ('that', np.float64(0.02390833188412863)), ('im', np.float64(0.023352415650268587)), ('me', np.float64(0.02258901514530549))]
Topic 1 [('restless', np.float64(0.19593502702573765)), ('is', np.float64(0.04754924843518879)), ('the', np.float64(0.045589642151815)), ('sleep', np.float64(0.04293826957756743)), ('to', np.float64(0.04058148556220812)), ('you', np.float64(0.03608746470369772)), ('im', np.float64(0.034083271194709636)), ('and', np.float64(0.03368407395404549)), ('why', np.float64(0.03364680088865664)), ('of', np.float64(0.029212376161578096))]


In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words, ngram_range=(1,2))

In [50]:
from bertopic import BERTopic

topic_model = BERTopic(
    vectorizer_model=vectorizer,
    min_topic_size=100,      # try 50, 80, 100; pick the one that gives 8–12 coherent topics
    n_gram_range=(1, 2),
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(stressed_posts, embeddings)  # pass precomputed embeddings!
topic_info = topic_model.get_topic_info()
topic_info.head(10)

2025-09-02 23:34:55,355 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-02 23:34:56,524 - BERTopic - Dimensionality - Completed ✓
2025-09-02 23:34:56,525 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-02 23:34:56,715 - BERTopic - Cluster - Completed ✓
2025-09-02 23:34:56,717 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-02 23:34:57,357 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,145,-1_rabies_im_anxiety_hiv,"[rabies, im, anxiety, hiv, get, got, cat, wate...",[Anxious about Rabies even though there's no r...
1,0,5459,0_im_like_anxiety_ive,"[im, like, anxiety, ive, feel, dont, get, know...",[Breaking up with boyfriend of two years becau...
2,1,315,1_restless_sleep_im_restlessness,"[restless, sleep, im, restlessness, restless r...","[very restless, Why are you so restless?, Why ..."


In [51]:
# Top words per topic
for tid in topic_info["Topic"].head(10):
    if tid == -1:
        continue
    print("Topic", tid, topic_model.get_topic(tid)[:10])  # list of (word, score)

Topic 0 [('im', np.float64(0.03699602286883397)), ('like', np.float64(0.024027532751498146)), ('anxiety', np.float64(0.021893767528781836)), ('ive', np.float64(0.019535979494089244)), ('feel', np.float64(0.01891336784662727)), ('dont', np.float64(0.01770210523506122)), ('get', np.float64(0.01694521407675328)), ('know', np.float64(0.015670082353035433)), ('time', np.float64(0.014691924924869558)), ('really', np.float64(0.0135603672527535))]
Topic 1 [('restless', np.float64(0.33758241519230286)), ('sleep', np.float64(0.06355385974379547)), ('im', np.float64(0.053996537553237085)), ('restlessness', np.float64(0.05107649542963644)), ('restless restless', np.float64(0.04610066971670003)), ('dont', np.float64(0.0424728493335089)), ('heart', np.float64(0.041595963445022734)), ('feeling', np.float64(0.04029495534822423)), ('like', np.float64(0.036627786254983356)), ('im restless', np.float64(0.036366385567389636))]


Not great performance. Try a better performing model. 

In [52]:
from sentence_transformers import SentenceTransformer

sbert = SentenceTransformer("all-mpnet-base-v2")
embeddings = sbert.encode(stressed_posts, normalize_embeddings=True, show_progress_bar=True)

Batches: 100%|██████████| 185/185 [01:38<00:00,  1.88it/s]


In [53]:
topic_model = BERTopic(
    min_topic_size=100,      # try 50, 80, 100; pick the one that gives 8–12 coherent topics
    n_gram_range=(1, 2),
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(stressed_posts, embeddings)  # pass precomputed embeddings!
topic_info = topic_model.get_topic_info()
topic_info.head(10)

2025-09-03 00:05:12,997 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-03 00:05:14,294 - BERTopic - Dimensionality - Completed ✓
2025-09-03 00:05:14,295 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-03 00:05:14,496 - BERTopic - Cluster - Completed ✓
2025-09-03 00:05:14,501 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-03 00:05:15,267 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,213,-1_the_and_to_my,"[the, and, to, my, it, of, in, was, that, but]",[Uncontrollable HIV anxiety New to Reddit so I...
1,0,5098,0_and_to_the_my,"[and, to, the, my, of, it, in, that, im, me]","[A story about my anxiety in the recent past, ..."
2,1,608,1_restless_you_im_nervous,"[restless, you, im, nervous, the, is, to, why,...","[Why are you so restless, Restless and restles..."


UMAP: squashes high-dimensional embeddings (384 or 768 dims) down to a smaller space (like 5–10 dims). This makes clustering easier and can emphasize local structure.

HDBSCAN: does the actual clustering on that reduced space.

In [54]:
from umap import UMAP
umap_model = UMAP(
    n_neighbors=8,       # more local clusters
    n_components=10,     # preserve more info
    min_dist=0.05,
    metric="cosine",
    random_state=42
)

In [ ]:
import hdbscan
hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=30,   # start here; try 30–60
    min_samples=10,
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True   # REQUIRED for probabilities
)

In [62]:
vectorizer = CountVectorizer(stop_words="english", ngram_range=(1,2), min_df=5, max_df=0.3)

topic_model = BERTopic(
    embedding_model=sbert,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    calculate_probabilities=True,
    verbose=True
)
topics, probs = topic_model.fit_transform(stressed_posts, embeddings)  # pass precomputed embeddings!
topic_info = topic_model.get_topic_info()
topic_info.head(20)

2025-09-03 00:48:35,654 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-03 00:48:39,363 - BERTopic - Dimensionality - Completed ✓
2025-09-03 00:48:39,363 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-03 00:48:39,623 - BERTopic - Cluster - Completed ✓
2025-09-03 00:48:39,625 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-09-03 00:48:39,939 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2216,-1_diabetes_urine_kidney_sinus,"[diabetes, urine, kidney, sinus, alot, shortne...",[Anxiety has ruined my life... I need some adv...
1,0,300,0_restless_restlessness_want sleep_oh god,"[restless, restlessness, want sleep, oh god, d...","[Restless ha ha, why are you always restless?,..."
2,1,250,1_abuse_police_bf_ex,"[abuse, police, bf, ex, loves, door, dating, s...","[After I took care of healing my self, my bigg..."
3,2,222,2_cardiologist_beats_pulse_ecg,"[cardiologist, beats, pulse, ecg, ekg, monitor...",[Why can't I believe what the doctors tell me?...
4,3,198,3_zoloft_dose_mg_lexapro,"[zoloft, dose, mg, lexapro, xanax, ssris, ssri...",[Advice greatly needed (26 year old female) so...
5,4,177,4_rent_homeless_bills_assistance,"[rent, homeless, bills, assistance, income, ur...",[My stress just got 10x worse I 20(M) am about...
6,5,172,5_dating_ex_friendship_gift,"[dating, ex, friendship, gift, gf, asks, messa...",[Anxiety? Stress? Guilt? Scarred? I’ll try to ...
7,6,165,6_brain tumor_aneurysm_sneeze_glasses,"[brain tumor, aneurysm, sneeze, glasses, left ...","[Brain tumor anxiety Hey everyone, I'm new to ..."
8,7,139,7_abuse_abused_memories_sexually,"[abuse, abused, memories, sexually, sexual, de...",[If you’re wondering whether adverse childhood...
9,8,139,8_stool_colon_colon cancer_bowel,"[stool, colon, colon cancer, bowel, diarrhea, ...",[Colon Cancer symptoms for 1.5 months Hello gu...


In [63]:
topic_info.shape

(43, 5)

In [64]:
# Top words per topic
for tid in topic_info["Topic"]:
    if tid == -1:
        continue
    print("Topic", tid, topic_model.get_topic(tid)[:])  # list of (word, score)

Topic 0 [('restless', np.float64(0.7305335626770151)), ('restlessness', np.float64(0.15945131009492622)), ('want sleep', np.float64(0.0575891080429339)), ('oh god', np.float64(0.038392738695289266)), ('default', np.float64(0.02964587853313813)), ('dancing', np.float64(0.024018798832478174)), ('agitated', np.float64(0.024018798832478174)), ('lazy', np.float64(0.022928821566252144)), ('pls', np.float64(0.022616991120434606)), ('huh', np.float64(0.0222344088998536))]
Topic 1 [('abuse', np.float64(0.03346826642417592)), ('police', np.float64(0.030986322513645796)), ('bf', np.float64(0.0289149067509353)), ('ex', np.float64(0.027568828477456304)), ('loves', np.float64(0.025429821601908403)), ('door', np.float64(0.021733651367253724)), ('dating', np.float64(0.021311361088249176)), ('son', np.float64(0.019672025619922318)), ('abusive', np.float64(0.016653692886748547)), ('yelling', np.float64(0.01620623486381729))]
Topic 2 [('cardiologist', np.float64(0.04275530782039152)), ('beats', np.float6

## ^ Gotta tweak the above hyperparameters more to get better topics!